In [ ]:
from PIL import Image, ImageOps
import cv2
import json
import numpy as np

import matplotlib.pyplot as plt
import random
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

img_path = '../OCR_data/output/'

In [142]:
def PIL_img_show(img):
    Image.fromarray(img).show()

def contours_filter(contours, hierarchy):#汉字的一个特点是文字内部有空洞，由此形成连通域，但是检测句子所在行的话应该去掉这部分
    first_child = hierarchy[0][0][2]
    next_child = first_child
    all_children=[]
    while(next_child!=-1):
        all_children.append(next_child)
        next_child = hierarchy[0][next_child][0]
    contours_filtered = [contours[x] for x in all_children]
    
#     print(len(all_children), all_children)
    return contours_filtered

def get_text_rate(bin_img, boxs_horizontal):
    
    img_heigth, img_width = bin_img.shape
    small_point_cnt=0
    large_not_text_cnt=0
    small_not_text_cnt=0
    all_text_str=[]
    all_ROI_area_rate=[]
    text_rate_metric=0

    for idx, box_horizontal in enumerate(boxs_horizontal):
        x1,y1=box_horizontal[0]
        x2,y2=box_horizontal[2]
        img_ROI = bin_img[y1:y2, x1:x2]
        ROI_area = (x2-x1)*(y2-y1)
        ROI_area_rate = ROI_area/img_heigth/img_width


        if ROI_area/img_heigth/img_width<0.0005:#是小的噪点，不是中文或字母
            small_point_cnt+=1
        elif ((sum(sum(1-img_ROI/255)))/ROI_area)>0.7:#黑色部分太大，属于大片黑色非文字区域
            large_not_text_cnt+=1
        else:
            text_str = pytesseract.image_to_string(Image.fromarray(img_ROI), lang='chi_sim')
            if text_str=='' or text_str ==' ' or text_str=='\t':
                small_not_text_cnt+=1
            else:
                #print(idx, text_str, ROI_area_rate, ROI_area_rate)
                all_text_str.append(text_str)
                all_ROI_area_rate.append(ROI_area_rate)
    print(small_point_cnt, large_not_text_cnt, small_not_text_cnt)

    
    ROI_area_weights=np.array(all_ROI_area_rate/sum(all_ROI_area_rate))
    all_test_length=[len(x) for x in all_text_str]
    length_div_area=np.array(all_test_length)/np.array(all_ROI_area_rate)

    text_rate_metric = np.mean((length_div_area-length_div_area.mean())**2*ROI_area_weights)
    return text_rate_metric

#     for i in range(len(all_text_str)):
#         text_rate_metric+=len(all_text_str[i])/all_ROI_area_rate[i]

#     return text_rate_metric/len(all_text_str)

def get_likely_text_area(bin_img):
    
    img_height, img_width = bin_img.shape[:2]
    
    kernel_size = (int(img_height/400), int(img_width/1000))
    kernel = np.ones(kernel_size,np.uint8)
    dilate_img = cv2.dilate(bin_img,kernel,iterations = 1)
#     PIL_img_show(dilate_img)

    kernel_size = (int(img_height/400), int(img_width/80))
    kernel = np.ones(kernel_size,np.uint8)
    erosion_img = cv2.erode(dilate_img,kernel,iterations = 2)
    
    image, contours, hierarchy = cv2.findContours(erosion_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#CHAIN_APPROX_SIMPLE

#     tmp_img = erosion_img-erosion_img
#     cv2.drawContours(tmp_img,contours,-1,(100,100,100),5)
#     PIL_img_show(tmp_img)

#     print(len(contours))
    contours_filtered = contours_filter(contours, hierarchy)
#    print(len(contours_filtered))
    areas=[]
    rects=[]#矩形信息包括：矩形中心，高度，宽度，旋转角
    boxs=[]#矩形信息包括：矩形四个点的坐标，经过rects信息计算而来，经过了计算把浮点数转化为图像坐标，矩形可以不是水平的。

    for cnt in contours_filtered:
        area = cv2.contourArea(cnt)
        areas.append(area)

        rect = cv2.minAreaRect(cnt) # 得到最小外接矩形的（中心(x,y), (宽,高), 旋转角度）
        rects.append(rect)

        box = cv2.boxPoints(rect) # cv2.boxPoints(rect) for OpenCV 3.x 获取最小外接矩形的4个顶点
        box = np.int0(box)
        boxs.append(box)

#     tmp_img = erosion_img-erosion_img
#     cv2.drawContours(tmp_img,boxs,-1,(100,100,100),5)
#     PIL_img_show(tmp_img)
    
    boxs_horizontal=[]#将boxs化为方向水平，但是如果句子是倾斜的则圈出的范围过于宽泛。
    for box in boxs:
        x1=min(box[0][0], box[1][0], box[2][0], box[3][0])
        x2=max(box[0][0], box[1][0], box[2][0], box[3][0])
        y1=min(box[0][1], box[1][1], box[2][1], box[3][1])
        y2=max(box[0][1], box[1][1], box[2][1], box[3][1])
        boxs_horizontal.append(np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2]]))

#     print(len(boxs_horizontal))
    return boxs_horizontal, boxs

def dilate_and_erode(bin_img):
    img_height, img_width = bin_img.shape[:2]
    
    kernel_size = (int(img_height/400), 0)
    kernel = np.ones(kernel_size,np.uint8)
    dilate_img = cv2.dilate(bin_img,kernel,iterations = 1)
#     PIL_img_show(dilate_img)

    kernel_size = (int(img_height/400), int(img_width/80))
    kernel = np.ones(kernel_size,np.uint8)
    erosion_img = cv2.erode(dilate_img,kernel,iterations = 2)
    
    return erosion_img

In [143]:
for img_id in range(42):
    img_name = 'img_'+str(1000000+img_id)+'.png'
    img = cv2.imread(img_path + img_name)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img_heigth, img_width = gray_img.shape[:2]
    
    bin_threshold, bin_img = cv2.threshold(gray_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
#     PIL_img_show(erosion_img)
    boxs_horizontal, boxs = get_likely_text_area(bin_img)
    text_rate_metric = get_text_rate(bin_img, boxs_horizontal)
    print(img_id, text_rate_metric)

4 0 1
0 249840.4517783487
6 3 2
1 47916.27646977664
9 3 5
2 50963.19897738599
10 3 4
3 21436.799812854297
6 3 2
4 21461.09754033595
15 3 1
5 61087.228558036586
6 1 6
6 70209.54529182124
3 1 9
7 65608.3676154649
5 1 8
8 33852.647031370936
6 1 5
9 100318.26252107139
2 1 7
10 26597.58037824231
1 0 1
11 59037.333999206196
2 1 6
12 181419.5938732705
4 1 5
13 6484.143413177878
7 0 5
14 261160.1906568356
3 0 10
15 59905.59910113714
32 1 3
16 77212.85318550986
68 0 17
17 575806.1264389001
8 0 2
18 13325.455336898573
8 0 1
19 38368.2520897021
3 0 4
20 21121.85222747711
8 0 1
21 13908.388155154087
12 3 3
22 31427.4614190713
7 3 3
23 11878.25693030198
5 3 7
24 17917.742342063917
25 3 3
25 65205.83963195457
7 0 7
26 334502.67840087
3 2 9
27 58205.04480135725
9 1 2
28 392851.3887511138
8 1 8
29 134423.70190482546
30 1 17
30 311969.8501063694
5 1 12
31 423889.8721642086
16 1 6
32 1221089.711201047
6 1 5
33 117359.93967368717
3 1 4
34 123144.29975053803
34 1 9
35 246805.79921082308
11 1 21
36 67972.8

In [126]:
img = cv2.imread(img_path + 'img_'+str(1000017)+'.png')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img_heigth, img_width = gray_img.shape[:2]
print(img_heigth, img_width)


bin_threshold, bin_img = cv2.threshold(gray_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
# PIL_img_show(gray_img)
PIL_img_show(bin_img)

kernel_size = (int(img_heigth/400), int(img_width/1000))
kernel = np.ones(kernel_size,np.uint8)
dilate_img = cv2.dilate(bin_img,kernel,iterations = 1)
PIL_img_show(dilate_img)

kernel_size = (int(img_heigth/400), int(img_width/80))
kernel = np.ones(kernel_size,np.uint8)
erosion_img = cv2.erode(dilate_img,kernel,iterations = 2)
PIL_img_show(erosion_img)

3462 2446


In [127]:
#image,contours,hierarchy=cv2.findContours(erosion_img,1,2)

# cv2.RETR_EXTERNAL表示只检测外轮廓
# cv2.RETR_LIST检测的轮廓不建立等级关系
# cv2.RETR_CCOMP建立两个等级的轮廓，上面的一层为外边界，里面的一层为内孔的边界信息。如果内孔内还有一个连通物体，这个物体的边界也在顶层。
# cv2.RETR_TREE建立一个等级树结构的轮廓。
image, contours, hierarchy = cv2.findContours(erosion_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#CHAIN_APPROX_SIMPLE

tmp_img = erosion_img-erosion_img
cv2.drawContours(tmp_img,contours,-1,(100,100,100),5)
PIL_img_show(tmp_img)

print(len(contours))
contours_filtered = contours_filter(contours, hierarchy)
print(len(contours_filtered))
areas=[]
rects=[]#矩形信息包括：矩形中心，高度，宽度，旋转角
boxs=[]#矩形信息包括：矩形四个点的坐标，经过rects信息计算而来，经过了计算把浮点数转化为图像坐标，矩形可以不是水平的。

for cnt in contours_filtered:
    area = cv2.contourArea(cnt)
    areas.append(area)
    
    rect = cv2.minAreaRect(cnt) # 得到最小外接矩形的（中心(x,y), (宽,高), 旋转角度）
    rects.append(rect)
    
    box = cv2.boxPoints(rect) # cv2.boxPoints(rect) for OpenCV 3.x 获取最小外接矩形的4个顶点
    box = np.int0(box)
    boxs.append(box)

tmp_img = erosion_img-erosion_img
cv2.drawContours(tmp_img,boxs,-1,(100,100,100),5)
PIL_img_show(tmp_img)


135
131


In [128]:
boxs_horizontal=[]#将boxs化为方向水平，但是如果句子是倾斜的则圈出的范围过于宽泛。
for box in boxs:
    x1=min(box[0][0], box[1][0], box[2][0], box[3][0])
    x2=max(box[0][0], box[1][0], box[2][0], box[3][0])
    y1=min(box[0][1], box[1][1], box[2][1], box[3][1])
    y2=max(box[0][1], box[1][1], box[2][1], box[3][1])
    boxs_horizontal.append(np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2]]))
    
print(len(boxs_horizontal))

cv2.drawContours(gray_img,boxs_horizontal,-1,(100,100,100),5)
PIL_img_show(gray_img)

131


In [131]:
small_point_cnt=0
large_not_text_cnt=0
small_not_text_cnt=0
all_text_str=[]
all_ROI_area_rate=[]
text_rate_metric=0

for idx, box_horizontal in enumerate(boxs_horizontal):
    x1,y1=box_horizontal[0]
    x2,y2=box_horizontal[2]
    img_ROI = bin_img[y1:y2, x1:x2]
    ROI_area = (x2-x1)*(y2-y1)
    ROI_area_rate = ROI_area/img_heigth/img_width
    
    
    if ROI_area/img_heigth/img_width<0.0005:#是小的噪点，不是中文或字母
        small_point_cnt+=1
    elif ((sum(sum(1-img_ROI/255)))/ROI_area)>0.7:#黑色部分太大，属于大片黑色非文字区域
        large_not_text_cnt+=1
    else:
        text_str = pytesseract.image_to_string(Image.fromarray(img_ROI), lang='chi_sim')
        if text_str=='' or text_str ==' ' or text_str=='\t':# (len(text_str)<=3 and '\t' in text_str):
            small_not_text_cnt+=1
        else:
            #print(idx, text_str, ROI_area_rate, ROI_area_rate)
            all_text_str.append(text_str)
            all_ROI_area_rate.append(ROI_area_rate)
print(small_point_cnt, large_not_text_cnt, small_not_text_cnt)

ROI_area_weights=np.array(all_ROI_area_rate/sum(all_ROI_area_rate))
all_test_length=[len(x) for x in all_text_str]
length_div_area=np.array(all_test_length)/np.array(all_ROI_area_rate)

text_rate_metric = np.mean((length_div_area-length_div_area.mean())**2*ROI_area_weights)
print(text_rate_metric)

# for i in range(len(all_text_str)):
#     text_rate_metric+=len(all_text_str[i])/all_ROI_area_rate[i]*ROI_area_weights[i]

# print(text_rate_metric/len(all_text_str))
print(all_text_str)

68 0 17
575806.1264389001
['(" M噩', 'spir0_ab "】 S/N 312578', '所用仪器', '1 肺通气功能正常', '结论', "BfPS 1′101 23 'c 73.4 ”F", 'FEF2575', "FEV1'FVC", 'FEV1′VC', 'FEV1', "2.52\n2.06\n81'5\n81′5\n5'51\n2′21\n\n50\n600\n\n252", '1.34\n084\n\n62′7\n096\n066\n111\n3.93\n50', '97\n94\n\n97\n59\n83\n1 1 2\n53', "2'45\n1.94\n\n79.2\n3'27\n1 .82\n56\n3^2o\n110", '2_17\n1.58\n\n72-8\n2.69\n1.29\n\n74\n3.12', "Z'sc。re", '%预 计', 'POST攀1', '%预 计', '%激发', '前阆墅 旦期 2018-01-芝6 08:33:4O', '曹堇 间', "3'27 _", '` 已预测的 - Hdng Kong Thoracic Society', 'FEV1% %', "79'20", 'FEV1 L', '扁 %预计 Z-霾c。re', '后 %瀛发', '全部曲线之最佳值', '正常肺量计', '评估', '患者组', '吸烟', '包-年', '21.36', '出生日期', '童量` kg', '高度 cm', '性别', '病人号码', '年龄', '应诊 日 期 20′1喹吐-26', '胍胁唰胁愤\n\nShenzh鸳n Hosp‖藿【 酥 sou【h髦m M蛐ka[ un岫脂‖y', "是 绯 梳员蒯睿量和容蜀时间\n\nf/誓堇帘i丨l|医脘\n\n暴fsou【hem Me傣kaLun岫脂‖y\n\n \n\n年龄\n性别\n高度 cm\n卜 重量 kg\nBM| 21.36\n包-年\n\n__=鳕蘸-{盗勒t萝胭_鬣__\n\n鞠\n\n \n\n \n\n菌 %预计 Z-ac。re 后 %滇发\n\n2'45 97\n1 '94 94\n79'20 97\n3'27 ' 59\n\n \n\n0 1 2 3 窜墨… 5 6 了", '肺功能测验结埕]']


In [130]:
length_div_area=np.array(all_test_length)/np.array(all_ROI_area_rate)
print(length_div_area)
print(ROI_area_weights)
print(length_div_area.mean())

[ 7613.69976359  4942.08951407  3450.6375     11451.00444444
  3790.17391304  3304.30403557  2049.15906681  3154.86857143
  3209.89534884  2752.64529915  3209.89534884  1908.50074074
  4675.05725806  4202.29865893]
[0.01949466 0.0210232  0.01720569 0.020739   0.0587413  0.02245957
 0.63016603 0.0235234  0.02312014 0.0269607  0.02312014 0.0933255
 0.00952458 0.01059609]
4265.302104536537


In [88]:
x=[]
for idx, box_horizontal in enumerate(boxs_horizontal):
    x1,y1=box_horizontal[0]
    x2,y2=box_horizontal[2]
    x.append((x2-x1)*(y2-y1))
print(np.sort(x)/img_heigth/img_width)

[0.00011337 0.00011337 0.00011337 0.00012966 0.00013179 0.00030113
 0.00045087 0.00045087 0.00045347 0.00046055 0.00046481 0.00049386
 0.00056684 0.00056684 0.00068493 0.00070855 0.00071705 0.00072626
 0.00073689 0.00074303 0.00079499 0.00079829 0.00080101 0.00083254
 0.00084742 0.00086088 0.00087812 0.00088001 0.00088993 0.00090127
 0.0009034  0.00091166 0.00091509 0.00094048 0.00094898 0.00101558
 0.0010366  0.00104298 0.00105148 0.00106849 0.00107002 0.00110155
 0.00120158 0.00121161 0.00133632 0.00133868 0.00134482 0.00135096
 0.00135816 0.00136608 0.00137068 0.00137694 0.00137694 0.00138946
 0.00139017 0.00139123 0.00139938 0.0014093  0.00143693 0.00143953
 0.00149869 0.00154132 0.0016352  0.00172554 0.00183017 0.00197046
 0.00198747 0.00214985 0.0021554  0.00240634 0.00255714 0.00257792
 0.00290327 0.02399666 0.02554484 0.02631892]


In [50]:
print(contours[areas.index(max(areas))])
print(areas.index(max(areas)))

areas_sorted = areas.copy()
areas_sorted.sort()
areas_sorted = [x/(img_heigth*img_width)*100 for x in areas_sorted]
if areas_sorted[-1]>99.5:# to remove the largest contours if the largest is the whole image.
    print('remove the largest one if it is the whole image')
    areas_sorted = areas_sorted[:-1]
print(areas_sorted[-10:])

s=0
for x in range(img_heigth):
    for y in range(img_width):
        s+=erosion_img.item(x,y)
print(sum(areas_sorted), 1-s/img_heigth/img_width, img_heigth, img_width)

[[[   0    0]]

 [[   0 2394]]

 [[  24 2394]]

 [[  25 2395]]

 [[  25 2405]]

 [[  23 2407]]

 [[  23 2410]]

 [[  22 2411]]

 [[   0 2411]]

 [[   0 3421]]

 [[2155 3421]]

 [[2155 3413]]

 [[2156 3412]]

 [[2198 3412]]

 [[2199 3413]]

 [[2199 3421]]

 [[2225 3421]]

 [[2225    0]]]
0
remove the largest one if it is the whole image
[0.639925423098675, 0.6767425826124811, 0.6840351239246291, 0.7258408805556562, 0.7710598878458345, 0.8208408359208399, 0.8311134076161699, 0.8620755294608167, 0.9061130269074427, 1.1340197117851143]
16.995316757537903 -211.80751156698136 3422 2226


In [136]:
cv2.aruco_CharucoBoard()
retval=cv2.aruco_CharucoBoard.getChessboardSize(bin_img)

TypeError: descriptor 'getChessboardSize' requires a 'cv2.aruco_CharucoBoard' object but received a 'numpy.ndarray'


textdetection.py
       A demo script of the Extremal Region Filter algorithm described in:
       Neumann L., Matas J.: Real-Time Scene Text Localization and Recognition, CVPR 2012

Extracting Class Specific Extremal Regions from 9 channels ...
    (...) this may take a while (...)
1
1
1
1
0
0
0
0
0


[[2027 3323]
 [2088 3323]
 [2088 3341]
 [2027 3341]]
2027 2088 3323 3341
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 254 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


[1, 2, 1, 2, 1, 2]
